In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    recall_score, precision_score, f1_score
)
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
import joblib

In [4]:
df = pd.read_csv("file.csv", low_memory=False)

print(f"\nDataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df


Dataset loaded: 42790 rows, 41 columns


,Date,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,1900-01-01,3102683.0,(2001 XX4),22.10,0.101054,0.225964,101.054342,225.964377,0.062792,0.140408,...,127.074771,368.799662,0.446121,186.829730,1.566791,2.460307e+06,255.653949,0.976140,J2000,True
1,1900-01-01,3285299.0,(2005 OE3),20.30,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,...,321.032500,716.317846,0.895238,58.408586,2.238299,2.460091e+06,55.020372,0.502570,J2000,True
2,1900-01-01,3653973.0,(2013 WU45),20.37,0.224158,0.501233,224.158379,501.233372,0.139286,0.311452,...,36.158624,621.104718,0.585125,153.791066,2.264194,2.460421e+06,231.962485,0.579612,J2000,False
3,1900-01-01,3703077.0,(2014 YT34),24.70,0.030518,0.068240,30.517923,68.240151,0.018963,0.042402,...,285.381728,442.772170,0.955427,240.118639,1.318368,2.460183e+06,13.910335,0.813059,J2000,False
4,1900-01-01,3771605.0,(2017 EE23),21.80,0.116026,0.259442,116.025908,259.441818,0.072095,0.161210,...,149.163448,299.267765,0.542113,244.187323,1.209095,2.460128e+06,87.787093,1.202936,J2000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42785,1927-04-25,3879296.0,(2019 UO2),25.80,0.018389,0.041119,18.388867,41.118757,0.011426,0.025550,...,205.813295,287.925666,0.629720,31.139610,1.076957,2.460113e+06,109.248767,1.250323,J2000,False
42786,1927-04-25,3746448.0,(2016 EO85),19.10,0.402305,0.899580,402.304580,899.580388,0.249980,0.558973,...,212.735581,1554.207369,1.207668,31.118780,4.044077,2.460723e+06,239.031712,0.231629,J2000,False
42787,1927-04-25,3443807.0,(2008 YX32),20.85,0.179703,0.401828,179.702855,401.827799,0.111662,0.249684,...,84.729106,500.025507,0.750929,149.925112,1.714879,2.459956e+06,176.146306,0.719963,J2000,False
42788,1927-04-25,3143081.0,(2002 VR85),20.30,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,...,204.441659,892.217185,0.718459,299.323640,2.909075,2.459812e+06,156.808351,0.403489,J2000,True


In [7]:
leakage_keywords = [
    # Direct thresholds
    'absolute magnitude',
    'moid',
    'minimum orbit intersection',

    # Derived from Absolute Magnitude (H = 15.618 - 2.5*log10(diameter))
    'est dia',
    'estimated dia',
    'diameter',

    # Related to MOID
    'miss dist',
    'close approach',
]

# Administrative columns
admin_keywords = [
    'date',
    'id',
    'name',
    'ref',
    'epoch',
    'equinox',
]
print("\nScanning all columns...")
leakage_cols = []
admin_cols = []
for col in df.columns:
    col_lower = col.lower()

    # Check for leakage
    for keyword in leakage_keywords:
        if keyword in col_lower:
            leakage_cols.append(col)
            print(f"{col}")
            break

    # Check for admin
    else:
        for keyword in admin_keywords:
            if keyword in col_lower:
                admin_cols.append(col)
                break
print(f"\nLeakage features to remove: {len(leakage_cols)}")
print(f"Admin columns : {len(admin_cols)}")
#Admin columns are columns that contain metadata about the data rather than actual features


Scanning all columns...
Absolute Magnitude
Est Dia in KM(min)
Est Dia in KM(max)
Est Dia in M(min)
Est Dia in M(max)
Est Dia in Miles(min)
Est Dia in Miles(max)
Est Dia in Feet(min)
Est Dia in Feet(max)
Close Approach Date
Epoch Date Close Approach
Miss Dist.(Astronomical)
Miss Dist.(lunar)
Miss Dist.(kilometers)
Miss Dist.(miles)
Minimum Orbit Intersection

Leakage features to remove: 16
Admin columns : 7


In [14]:

#DEDUPLICATING BY OBJECT
object_id_column = 'Neo Reference ID'
target_column = 'Hazardous'

print(f"Before: {len(df)} rows, {df[object_id_column].nunique()} unique objects")

# Keep most recent observation per object
df['Date'] = pd.to_datetime(df['Date'])
df_sorted = df.sort_values([object_id_column, 'Date'], ascending=[True, False])
df_unique = df_sorted.groupby(object_id_column).first().reset_index()

print(f"After: {len(df_unique)} rows, {df_unique[object_id_column].nunique()} unique objects")

Before: 42790 rows, 11503 unique objects
After: 11503 rows, 11503 unique objects


In [16]:
#REMOVING LEAKAGE FEATURRES
all_cols_to_drop = list(set(leakage_cols + admin_cols))
all_cols_to_drop = [col for col in all_cols_to_drop if col in df_unique.columns and col != target_column]

print(f"\nRemoving {len(all_cols_to_drop)} columns:")
for col in sorted(all_cols_to_drop):
    print(f"{col}")

df_clean = df_unique.drop(columns=all_cols_to_drop)

print(f"\nRemaining columns: {df_clean.shape[1]}")
print("\nFeatures available for modeling:")
for col in df_clean.columns:
    if col != target_column:
        print(f"  ✓ {col}")


Removing 23 columns:
Absolute Magnitude
Close Approach Date
Date
Epoch Date Close Approach
Epoch Osculation
Equinox
Est Dia in Feet(max)
Est Dia in Feet(min)
Est Dia in KM(max)
Est Dia in KM(min)
Est Dia in M(max)
Est Dia in M(min)
Est Dia in Miles(max)
Est Dia in Miles(min)
Minimum Orbit Intersection
Miss Dist.(Astronomical)
Miss Dist.(kilometers)
Miss Dist.(lunar)
Miss Dist.(miles)
Name
Neo Reference ID
Orbit Determination Date
Orbit ID

Remaining columns: 18

Features available for modeling:
  ✓ Relative Velocity km per sec
  ✓ Relative Velocity km per hr
  ✓ Miles per hour
  ✓ Orbiting Body
  ✓ Orbit Uncertainty
  ✓ Jupiter Tisserand Invariant
  ✓ Eccentricity
  ✓ Semi Major Axis
  ✓ Inclination
  ✓ Asc Node Longitude
  ✓ Orbital Period
  ✓ Perihelion Distance
  ✓ Perihelion Arg
  ✓ Aphelion Dist
  ✓ Perihelion Time
  ✓ Mean Anomaly
  ✓ Mean Motion


In [17]:
#PREPARE DATA
X = df_clean.drop(columns=[target_column])
y = df_clean[target_column]

# Convert target to binary
if y.dtype == 'bool':
    y = y.astype(int)
elif y.dtype == 'object':
    y_map = {'True': 1, 'False': 0, True: 1, False: 0, 1: 1, 0: 0, 'true': 1, 'false': 0}
    y = y.map(y_map).fillna(0).astype(int)

print(f"\nFeatures: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")
print(f"Hazardous rate: {y.mean()*100:.2f}%")


Features: (11503, 17)
Target distribution: {0: 9683, 1: 1820}
Hazardous rate: 15.82%


In [18]:
#TRAIN-TEST SPLIT

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)} ({y_train.mean()*100:.1f}% hazardous)")
print(f"Test:  {len(X_test)} ({y_test.mean()*100:.1f}% hazardous)")

Train: 9202 (15.8% hazardous)
Test:  2301 (15.8% hazardous)


In [19]:
#FEATURE ENGENERING
def engineer_orbital_features(X_df):
    """Engineer features from pure orbital mechanics"""
    X_out = X_df.copy()

    # Orbital shape complexity
    if 'Eccentricity' in X_out.columns and 'Inclination' in X_out.columns:
        X_out['orbit_complexity'] = X_out['Eccentricity'] * X_out['Inclination']
        X_out['ecc_incl_ratio'] = X_out['Eccentricity'] / (X_out['Inclination'] + 0.1)

    # Orbital energy/velocity features
    if 'Relative Velocity km per sec' in X_out.columns:
        X_out['velocity_sq'] = X_out['Relative Velocity km per sec'] ** 2
        X_out['velocity_log'] = np.log1p(X_out['Relative Velocity km per sec'])

    # Semi-major axis features
    if 'Semi Major Axis' in X_out.columns:
        X_out['sma_log'] = np.log1p(X_out['Semi Major Axis'])
        # Earth's orbit is ~1 AU
        X_out['sma_earth_ratio'] = X_out['Semi Major Axis'] / 1.0

    # Perihelion-Aphelion dynamics
    if 'Perihelion Distance' in X_out.columns and 'Aphelion Dist' in X_out.columns:
        X_out['orbit_range'] = X_out['Aphelion Dist'] - X_out['Perihelion Distance']
        X_out['orbit_eccentricity_calc'] = (X_out['Aphelion Dist'] - X_out['Perihelion Distance']) / (X_out['Aphelion Dist'] + X_out['Perihelion Distance'] + 0.01)
        X_out['crosses_earth'] = ((X_out['Perihelion Distance'] < 1.017) & (X_out['Aphelion Dist'] > 0.983)).astype(int)

    # Orbital period features
    if 'Orbital Period' in X_out.columns:
        X_out['period_log'] = np.log1p(X_out['Orbital Period'])
        X_out['period_sq'] = X_out['Orbital Period'] ** 2

    # Jupiter Tisserand parameter (comet vs asteroid indicator)
    if 'Jupiter Tisserand Invariant' in X_out.columns:
        X_out['tisserand_sq'] = X_out['Jupiter Tisserand Invariant'] ** 2

    # Argument interactions
    if 'Perihelion Arg' in X_out.columns and 'Asc Node Longitude' in X_out.columns:
        X_out['orbital_angles_sum'] = X_out['Perihelion Arg'] + X_out['Asc Node Longitude']

    # Eccentricity powers
    if 'Eccentricity' in X_out.columns:
        X_out['ecc_sq'] = X_out['Eccentricity'] ** 2
        X_out['ecc_cube'] = X_out['Eccentricity'] ** 3

    return X_out

print("Engineering orbital features...")
X_train_eng = engineer_orbital_features(X_train)
X_test_eng = engineer_orbital_features(X_test)

print(f"Features after engineering: {X_train_eng.shape[1]}")


Engineering orbital features...
Features after engineering: 32


In [20]:
# Handle numerical columns only
numerical_cols = X_train_eng.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train_eng.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical: {len(numerical_cols)}, Categorical: {len(categorical_cols)}")

# Impute
num_imputer = SimpleImputer(strategy='median')
X_train_proc = pd.DataFrame(
    num_imputer.fit_transform(X_train_eng[numerical_cols]),
    columns=numerical_cols,
    index=X_train_eng.index
)
X_test_proc = pd.DataFrame(
    num_imputer.transform(X_test_eng[numerical_cols]),
    columns=numerical_cols,
    index=X_test_eng.index
)

# Handle categorical if any
if categorical_cols:
    from sklearn.preprocessing import LabelEncoder
    for col in categorical_cols:
        le = LabelEncoder()
        X_train_proc[col] = le.fit_transform(X_train_eng[col].astype(str))
        # Safely transform test labels: map unseen labels to -1 instead of failing
        mapping = {label: idx for idx, label in enumerate(le.classes_)}
        X_test_proc[col] = (
            X_test_eng[col].astype(str)
            .map(mapping)
            .fillna(-1)
            .astype(int)
        )

X_train = X_train_proc
X_test = X_test_proc


Numerical: 31, Categorical: 1


In [23]:
#HANDEL IMBALANCE
smote = SMOTE(random_state=42, k_neighbors=min(5, y_train.sum()-1))
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"After SMOTE: {pd.Series(y_train_balanced).value_counts().to_dict()}")


After SMOTE: {0: 7746, 1: 7746}


In [26]:
#SCALING
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_balanced),
    columns=X_train_balanced.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train_balanced.columns
)



In [28]:
#MODEL TRAINING
models = {
    'XGBoost': xgb.XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    ),
    'LightGBM': lgb.LGBMClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    'RandomForest': RandomForestClassifier(
        n_estimators=200,
        max_depth=7,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
}

results = []

for name, model in models.items():
    print(f"\n{name}:")
    model.fit(X_train_scaled, y_train_balanced)

    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    metrics = {
        'Model': name,
        'Recall': recall_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba),
        'Accuracy': (y_pred == y_test).mean()
    }
    results.append(metrics)

    print(f"  Recall:    {metrics['Recall']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  F1:        {metrics['F1']:.4f}")
    print(f"  ROC-AUC:   {metrics['ROC-AUC']:.4f}")

    best_model = None
best_score = -1
best_model_name = None

for name, model in models.items():
    print(f"\n{name}:")

    # Train
    model.fit(X_train_scaled, y_train_balanced)

    # Predict
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Metrics
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    acc = (y_pred == y_test).mean()

    print(f"  Recall:    {recall:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  F1:        {f1:.4f}")
    print(f"  ROC-AUC:   {auc:.4f}")

    # 🔥 SELECT BEST MODEL (RECALL FIRST)
    if recall > best_score:
        best_score = recall
        best_model = model
        best_model_name = name

print("\n🏆 BEST MODEL SELECTED:", best_model_name)



XGBoost:
  Recall:    0.7308
  Precision: 0.4845
  F1:        0.5827
  ROC-AUC:   0.8917

LightGBM:
  Recall:    0.7418
  Precision: 0.4821
  F1:        0.5844
  ROC-AUC:   0.8926

RandomForest:
  Recall:    0.8599
  Precision: 0.3569
  F1:        0.5044
  ROC-AUC:   0.8571

XGBoost:
  Recall:    0.7308
  Precision: 0.4845
  F1:        0.5827
  ROC-AUC:   0.8917

LightGBM:
  Recall:    0.7418
  Precision: 0.4821
  F1:        0.5844
  ROC-AUC:   0.8926

RandomForest:
  Recall:    0.8599
  Precision: 0.3569
  F1:        0.5044
  ROC-AUC:   0.8571

🏆 BEST MODEL SELECTED: RandomForest


In [29]:
import joblib
import json

# Save model
joblib.dump(best_model, "sentinel_model.pkl")

# Save scaler
joblib.dump(scaler, "scaler.pkl")

# Save imputer
joblib.dump(num_imputer, "num_imputer.pkl")

# Save feature names
with open("feature_names.json", "w") as f:
    json.dump(X_train.columns.tolist(), f)

print("✅ Model and preprocessing artifacts saved!")
print("Files created:")
print(" - sentinel_model.pkl")
print(" - scaler.pkl")
print(" - num_imputer.pkl")
print(" - feature_names.json")


✅ Model and preprocessing artifacts saved!
Files created:
 - sentinel_model.pkl
 - scaler.pkl
 - num_imputer.pkl
 - feature_names.json


In [30]:
loaded_model = joblib.load("sentinel_model.pkl")

sample = X_test.iloc[:1]
sample_scaled = scaler.transform(sample)

prob = loaded_model.predict_proba(sample_scaled)[0, 1]
print("Test probability:", prob)


Test probability: 0.2421205723044931


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [31]:
from google.colab import files

files.download("sentinel_model.pkl")
files.download("scaler.pkl")
files.download("num_imputer.pkl")
files.download("feature_names.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>